In [1]:
import numpy as np
import cv2

a = np.zeros(25)
# b = a.reshape

In [2]:
a

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0.])

In [3]:
b = a.reshape(5,5)
b

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [4]:
b[2,2]=1
b[1,1]=1
b[3,3]=1
b[2,1]=2
b[1,2]=2
b

array([[0., 0., 0., 0., 0.],
       [0., 1., 2., 0., 0.],
       [0., 2., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0.]])

In [8]:
c = np.zeros(3*5*5)
c = c.reshape(3,5,5)
c[2,3,4] = 2
c

array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 2.],
        [0., 0., 0., 0., 0.]]])

In [9]:
y = np.where(c == 2)
y

(array([2], dtype=int64), array([3], dtype=int64), array([4], dtype=int64))

In [11]:
c[y] = 10
c

array([[[ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0., 10.],
        [ 0.,  0.,  0.,  0.,  0.]]])

In [ ]:
# data_augment.py
import cv2
import os
import numpy as np
import glob
import random
# from PIL import Image


# 椒盐噪声（输入图像，噪声个数）
def salt(img, n):
    for k in range(n):
        i = int(np.random.random() * img.shape[1])
        j = int(np.random.random() * img.shape[0])
        if img.ndim == 2:
            img[j, i] = 255
        elif img.ndim == 3:
            img[j, i, 0] = 255
            img[j, i, 1] = 255
            img[j, i, 2] = 255
    return img

# 椒盐噪声
def SaltAndPepper(src,percetage):
    SP_NoiseImg=src.copy()
    SP_NoiseNum=int(percetage*src.shape[0]*src.shape[1])
    for i in range(SP_NoiseNum):
        randR=np.random.randint(0,src.shape[0]-1)
        randG=np.random.randint(0,src.shape[1]-1)
        randB=np.random.randint(0,3)
        if np.random.randint(0,1)==0:
            SP_NoiseImg[randR,randG,randB]=0
        else:
            SP_NoiseImg[randR,randG,randB]=255
    return SP_NoiseImg

# 高斯噪声
def addGaussianNoise(image,percetage):
    G_Noiseimg = image.copy()
    w = image.shape[1]
    h = image.shape[0]
    G_NoiseNum=int(percetage*image.shape[0]*image.shape[1])
    for i in range(G_NoiseNum):
        temp_x = np.random.randint(0,h)
        temp_y = np.random.randint(0,w)
        G_Noiseimg[temp_x][temp_y][np.random.randint(3)] = np.random.randn(1)[0]
    return G_Noiseimg

# 昏暗
def darker(image,percetage=0.8):
    image_copy = image.copy()
    w = image.shape[1]
    h = image.shape[0]
    #get darker
    for xi in range(0,w):
        for xj in range(0,h):
            image_copy[xj,xi,0] = int(image[xj,xi,0]*percetage)
            image_copy[xj,xi,1] = int(image[xj,xi,1]*percetage)
            image_copy[xj,xi,2] = int(image[xj,xi,2]*percetage)
    return image_copy

# 亮度
def brighter(image, percetage=1.4):
    image_copy = image.copy()
    w = image.shape[1]
    h = image.shape[0]
    #get brighter
    for xi in range(0,w):
        for xj in range(0,h):
            image_copy[xj,xi,0] = np.clip(int(image[xj,xi,0]*percetage),a_max=255,a_min=0)
            image_copy[xj,xi,1] = np.clip(int(image[xj,xi,1]*percetage),a_max=255,a_min=0)
            image_copy[xj,xi,2] = np.clip(int(image[xj,xi,2]*percetage),a_max=255,a_min=0)
    return image_copy

# 旋转
def rotate(image, angle, center=None, scale=1.0):
    (h, w) = image.shape[:2]
    # If no rotation center is specified, the center of the image is set as the rotation center
    if center is None:
        center = (w / 2, h / 2)
    m = cv2.getRotationMatrix2D(center, angle, scale)
    rotated = cv2.warpAffine(image, m, (w, h))
    return rotated

# 翻转
def flip(image):
    flipped_image = np.fliplr(image)
    return flipped_image

class AUGMENT_data():
    def __init__(self, data_path):
        # 初始化函数，读取所有data_path下的图片
        self.data_path = data_path
        self.imgs_paths = glob.glob(os.path.join(data_path, 'imgs/*.jpg'))
        self.image_bright = 0
        self.image_path = ''
        self.label_path = ''
    
    def augment_img(self, img, lab):
        image = img
        label = lab
        # print(self.image_bright)
        #变亮、变暗
        if self.image_bright > 180:
            img_b = darker(image)
            save_imgres_path = self.image_path.split('.')[0] + '_darker.jpg'
            save_labelres_path = self.label_path.split('.')[0] + '_darker.png'
            cv2.imwrite(save_imgres_path, img_b)
            cv2.imwrite(save_labelres_path, label)
        elif self.image_bright < 100:
            img_b = brighter(image)
            save_imgres_path = self.image_path.split('.')[0] + '_bright.jpg'
            save_labelres_path = self.label_path.split('.')[0] + '_bright.png'
            cv2.imwrite(save_imgres_path, img_b)
            cv2.imwrite(save_labelres_path, label)

        # 旋转
        rotated_90 = rotate(image, 90)
        rotated_90_label = rotate(label, 90)
        save_imgres_path = self.image_path.split('.')[0] + '_rotated_90.jpg'
        save_labelres_path = self.label_path.split('.')[0] + '_rotated_90.png'
        cv2.imwrite(save_imgres_path, rotated_90)
        cv2.imwrite(save_labelres_path, rotated_90_label)

        rotated_180 = rotate(image, 180)
        rotated_180_label = rotate(label, 180)
        save_imgres_path = self.image_path.split('.')[0] + '_rotated_180.jpg'
        save_labelres_path = self.label_path.split('.')[0] + '_rotated_180.png'
        cv2.imwrite(save_imgres_path, rotated_180)
        cv2.imwrite(save_labelres_path, rotated_180_label)

        # 镜像
        flipCode = random.choice([-1, 0, 1])
        flipped_img = cv2.flip(image, flipCode)
        flipped_label = cv2.flip(label, flipCode)
        save_imgres_path = self.image_path.split('.')[0] + '_flipped.jpg'
        save_labelres_path = self.label_path.split('.')[0] + '_flipped.png'
        cv2.imwrite(save_imgres_path, flipped_img)
        cv2.imwrite(save_labelres_path, flipped_label)

        # 增加噪声
        img_salt = SaltAndPepper(img, 0.2)
        save_imgres_path = self.image_path.split('.')[0] + '_salt.jpg'
        save_labelres_path = self.label_path.split('.')[0] + '_salt.png'
        cv2.imwrite(save_imgres_path, img_salt)
        cv2.imwrite(save_labelres_path, label)

        img_gauss = addGaussianNoise(img, 0.2)
        save_imgres_path = self.image_path.split('.')[0] + '_gauss.jpg'
        save_labelres_path = self.label_path.split('.')[0] + '_gauss.png'
        cv2.imwrite(save_imgres_path, img_gauss)
        cv2.imwrite(save_labelres_path, label)

    def getitem(self):
        for image_path in self.imgs_paths:
            # 根据image_path生成label_path
            label_path = image_path.replace('imgs', 'masks')
            label_path = label_path.replace('jpg', 'png')
            self.image_path = image_path
            self.label_path = label_path
            # 读取训练图片和标签图片
            image = cv2.imread(image_path)
            label = cv2.imread(label_path)
            self.image_bright = image.mean()
            print(image.shape)
            # print(image.mean())
            # print(type(image))
            # print(image.max())
            # print(image.min())
            print(label.shape)
            # 进行数据增广，并保存
            image = self.augment_img(image, label)
            print('saved ********')
        

    def __len__(self):
        # 返回训练集大小
        return len(self.imgs_paths)


if __name__ == "__main__":
    augment_data = AUGMENT_data("data_augment/")
    print("数据个数：", len(augment_data))
    augment_data.getitem()
    print('end')


In [10]:
# 图像的亮度、对比度、色度和锐度四种方式的增强（或减弱）处理
import os
import numpy as np
from PIL import Image
from PIL import ImageEnhance

# 原始图像
def ImageAugument():
    path = r'C:\\Users\\hkc\\Desktop\\meter_img\\temporary_test'  # 文件夹目录
    files = os.listdir(path)  # 得到文件夹下的所有文件名称
    # 遍历文件夹
    prefix = path + '/'
    for file in files:
        # print(file)
        image = Image.open(prefix + file)
        # image.show()

        # 亮度增强
        enh_bri = ImageEnhance.Brightness(image)
        brightness = 1.5
        image_brightened = enh_bri.enhance(brightness)
        image_brightened.save(prefix + file.strip('.jpg') + '-lightup' + '.jpg')
        # 亮度减弱
        enh_bri = ImageEnhance.Brightness(image)
        brightness = 0.5
        image_brightened = enh_bri.enhance(brightness)
        image_brightened.save(prefix + file.strip('.jpg') + '-lightdown' + '.jpg')

        # 色度增强
        enh_col = ImageEnhance.Color(image)
        color = 1.5
        image_colored = enh_col.enhance(color)
        image_colored.save(prefix + file.strip('.jpg') + '-colorup' + '.jpg')
        # 色度减弱
        enh_col = ImageEnhance.Color(image)
        color = 0.5
        image_colored = enh_col.enhance(color)
        image_colored.save(prefix + file.strip('.jpg') + '-colordown' + '.jpg')

        # 对比度增强
        enh_con = ImageEnhance.Contrast(image)
        contrast = 1.7
        image_contrasted = enh_con.enhance(contrast)
        image_contrasted.save(prefix + file.strip('.jpg') + '-contrastup' + '.jpg')
        # 对比度减弱
        enh_con = ImageEnhance.Contrast(image)
        contrast = 0.7
        image_contrasted = enh_con.enhance(contrast)
        image_contrasted.save(prefix + file.strip('.jpg') + '-contrastdown' + '.jpg')

        # # 锐度增强
        # enh_sha = ImageEnhance.Sharpness(image)
        # sharpness = 3.0
        # image_sharped = enh_sha.enhance(sharpness)
        # image_sharped.save(prefix + file.strip('.jpg') + '-moreSharup' + '.jpg')
        # # 锐度减弱
        # enh_sha = ImageEnhance.Sharpness(image)
        # sharpness = 1.5
        # image_sharped = enh_sha.enhance(sharpness)
        # image_sharped.save(prefix + file.strip('.jpg') + '-moreShardown' + '.jpg')

        # 椒盐噪声
        SP_NoiseImg = image.copy()
        pim = SP_NoiseImg.load()
        percetage = 0.1
        SP_NoiseNum=int(percetage*image.width*image.height)
        for i in range(SP_NoiseNum):
            randR=np.random.randint(0,image.width-1)
            randG=np.random.randint(0,image.height-1)
            randB=np.random.randint(0,3)
            pim[randR,randG]=(0,0,0)
        SP_NoiseImg.save(prefix + file.strip('.jpg') + '-Noise' + '.jpg')

if __name__ == '__main__':
    ImageAugument()

In [5]:
from tqdm import tqdm
import time
 
for i in tqdm(range(100)):
  time.sleep(0.1)

100%|██████████| 100/100 [00:10<00:00,  9.81it/s]
